In [1]:
import re

class TreeNode:
    
    '''
     x(attr1)
    /\
    x(attr1=3) x(attr1=4)
    '''
    partitionAttrNum = 0#标记在当前node下，分割的attr是哪一个
    partitionAttrVal = 0#标记当前node的某个attr的具体值是多少，
    children=None#保存各种attr的node,leaf node check empty, if not l
    leafTag=None#如果是leaf，就赋对应tag的值
    ID=None
    par=None
    
    def __init__(self, attrVal):
        self.partitionAttrVal =attrVal 
        self.partitionAttrNum = 0#标记在当前node下，分割的attr是哪一个
        self.children={}
        self.ID=None
        self.par=None
        self.leafTag=None#如果是leaf，就赋对应tag的值
        
def preprocess():
    print "preprocess data: "
    inputList=[]
    tagSet=set()#统计tag有多少种
    for index, line in enumerate(open('/Users/shuofeng/Downloads/poker.train', 'r').readlines()):   
#         if index<10: #！bug free后测试全部数据集
        w=re.split(' |\n',line)  
        del w[-1]      #delete the new line character
            
        tagSet.add(w[0])
        
        for i in range(1,len(w)):
            idx=w[i].index(':')     #找到:所在的位置
            w[i]=w[i][idx+1:]       #提取:后面的attr val
        if w[0]=="":               #需要做该检查，因为文件结尾处有可能有若干个空行
            break
        w.append(index)        #把row number加入到entry中便于后续debug的时候知道是哪一行数据
        inputList.append(w)     #把处理好的数据加入到总的list of list中            
#     print inputList
    return inputList,tagSet

def testData():
    print "preprocess data: "
    testList=[]
    for index, line in enumerate(open('/Users/shuofeng/Downloads/poker.test', 'r').readlines()):   
#         if index<10: #！bug free后测试全部数据集
        w=re.split(' |\n',line)   
        del w[-1]
#             print w
        for i in range(1,len(w)):
            idx=w[i].index(':')     #找到:所在的位置
            w[i]=w[i][idx+1:]       #提取:后面的attr val
        if w[0]=="":               #需要做该检查，因为文件结尾处有可能有若干个空行
            break
        w.append(index)        #把row number加入到entry中便于后续debug的时候知道是哪一行数据
        """print w"""
        testList.append(w)            
#     print inputList
    return testList   
'''
@inputList list of list 注意list of list的下标是从0开始
@rowNums list of integers
'''
def curGini(inputList,rowNums):
    tagMap={}
    for rowNum in rowNums:
        curTag=inputList[rowNum][0]
        if curTag in tagMap.keys():
            tagMap[curTag]+=1.0
        else:
            tagMap[curTag]=1.0
    print "tagMap: ",
    print tagMap
    sum=0.0
    for i in tagMap.keys():
        sum+=pow(tagMap[i]/len(inputList),2)
#         print sum
    res=1-sum    
    print "current gini score: ",
    print res
    return res  



'''
@inputList list of list
@rowNums list
@availAttrNums string
@root current node
'''
def nextGini(inputList,rowNums,availAttrNums,root,level):
    print
    print "新的一个分支"
    print
    print "level of tree ",
    print level
#     print"root children:",
#     print root.children
#     print "当前节点的分割attribute:",
#     print root.partitionAttrNum
#     if root.par!=None:
#         print "root parent value:",
#         print root.par.partitionAttrVal
#     print "old root ID:",
#     print root.ID
    
    global nodeCount
    root.ID=nodeCount
    nodeCount+=1

#     print "new root ID:",
#     print root.ID
    
    print "row nums: ",
    print rowNums
    print "available attr numbers: ",
    print availAttrNums
    
    #递归终止条件,当rowNums list里面所有的数据的tag都一样，已经纯净了，purity=100%
    testSet=set()
    for rowNum in rowNums:
        curList=inputList[rowNum]
        curTag=curList[0]
        if curTag not in testSet:
            testSet.add(curTag)
    if len(testSet)==1:
        root.leafTag=inputList[rowNums[0]][0]
        print "root tag:",
        print root.leafTag
        return
    
    #非递归终止条件
    minSum=float("inf")
    chooseAttrNum=-1
    for attrNum in availAttrNums:#availAttrNums是一个string,记录当前branch还有哪些数据，目的是使各个branch互不影响
        attrNum=int(attrNum)
        curMap={}
        for rowNum in rowNums:
            curList=inputList[rowNum]#获得rowNum行的数据
            '''
            print curList
            print curList[attrNum]
            ''' 
            
            if curList[attrNum] in curMap.keys():#如果rowNum行数据的第n个attribute的具体值在map里，就在原来的list上追加该行的行数
                curMap[curList[attrNum]].append(rowNum)
            else:
                curMap[curList[attrNum]]=[]#如果不在，就新建一个list of list,同时记录下改行的行数
                curMap[curList[attrNum]].append(rowNum)
        print "list of map: ",
        print curMap
        
        #map的数据结构参看sublime伪代码,利用map把某一attribute的所有行保存在一起
        
        #计算某种分割的gini,minSum记录所有分割最小gini,chooseAttrNum记录最小gini对应的attribute number
        total=float(len(rowNums))#当前branch下的总行数
        sum=0.0

        for k,v in curMap.items():
            ratio=len(v)/total
            
            curTerm=ratio*curGini(inputList,v)#关键一步
            sum+=curTerm
        print "用当前上面的attr分割产生的新的gini score： ",
        print sum
        if sum<minSum:
            minSum=sum
            chooseAttrNum=attrNum

    
    root.partitionAttrNum=chooseAttrNum#当前node记录下用了哪个attribute分割
    print "当前节点的分割attribute:",
    print root.partitionAttrNum
    
    #没有gain怎么停止分割？
    
    #重新根据选中的attr建立map
    partitionMap={}
    for rowNum in rowNums:
        curList=inputList[rowNum]#获得rowNum行的数据
        print curList
        print curList[chooseAttrNum]
        if curList[chooseAttrNum] in partitionMap.keys():#如果rowNum行数据的第n个attribute的具体值在map里，就在原来的list上追加该行的行数
            partitionMap[curList[chooseAttrNum]].append(rowNum)
        else:
            partitionMap[curList[chooseAttrNum]]=[]#如果不在，就新建一个list of list,同时记录下改行的行数
            partitionMap[curList[chooseAttrNum]].append(rowNum)
    print "最终确定的partition map",
    print partitionMap
    
    
    tmp=str(chooseAttrNum)
    print "最终确定的partition attr num",
    print tmp
    newAvailNums=availAttrNums.replace(tmp,'')#标记选中的attribute为用过，用空字符替换attribute
    
    for k,v in partitionMap.items():
        
        newRowNums=v
        
        newRoot=TreeNode(k)
        newRoot.par=root
#         print "len before append",
#         print len(root.l)
        
        root.children[k]=newRoot
        print "分割的具体值",
        print k
#         print "len after append",
#         print len(root.l)
        
#         #child is of treeNode type
#         print "type of child:",
#         print type(root.l[len(root.l)-1])

#         #child is built succuessfully 
#         print "child ID:",
#         print root.l[len(root.l)-1].ID

#         #root and child are not the same       
#         if root is root.l[len(root.l)-1]:
#             print "error"
        nextGini(inputList,newRowNums,newAvailNums,newRoot,level+1)
# def test(test,root):
    
        

if __name__ == "__main__":
    global nodeCount
    nodeCount=0
    inputList,tagSet=preprocess()#list of list
    print "input list is: ",
    print inputList
#     curGiniScore=curGini(inputList,[0,1,2,3,4,5,6,7,8,9])
#     print curGiniScore
    


#     attrNums=[0,1,2,3,4,5,6,7,8,9]

#     availAttrNums=[i for i in range(1,len(inputList[0])-1)]
#     print availAttrNums
        
#     rowNums=[i for i in range(len(inputList))]
#     print rowNums
    
#     root=TreeNode(-1)
#     level=0   
    
#     nextGini(inputList,rowNums,availAttrNums,root,level)
    



# #     """
# #     pre order traversal
# #     """
# #     print
# #     print "遍历树"
# #     stack=[]
# #     stack.append(root)  
# #     while stack:
# #         cur=stack.pop()
# #         print "current ID:"
# #         print cur.ID        

# #         if not cur.children:
# #             print "leaf tag:",
# #             print cur.leafTag
# #             continue
# # #         print "current node's list length:"
# # #         print len(cur.children)
# #         for k,v in cur.children.items():
# # #             print type(k)
# #             stack.append(v)

#     '''
#     build confusion matrix
#     '''
#     outPutListList=[]
#     print "number of tags:",
#     print len(tagSet)
    
#     for i in range(len(tagSet)):
#         tmpList=[0 for j in range(len(tagSet))]
#         outPutListList.append(tmpList)
        
#     for i in outPutListList:
#         print i
        
    


#     testList=testData()#list of list
#     """
#     search the target value
#     """ 
#     print
#     print "搜索树"
#     trueCount=0
#     falseCount=0
#     for test in testList:
# #     test=[3,4,5,5,3,10]
    
#         stack=[]
#         stack.append(root)  
#         while stack:
#             cur=stack.pop()
#             print "current ID:"
#             print cur.ID    

#             testAttrValue=str(test[cur.partitionAttrNum])
#             print "test attr value"
#             print testAttrValue

#             if not cur.children:
#                 print "leaf tag:",
#                 print cur.leafTag
                
#                 outPutListList[int(test[0])-1][int(cur.leafTag)-1]+=1
                
#                 if cur.leafTag==test[0]:
#                     print "true"
#                     trueCount+=1
#                 else:
#                     print "false"
#                     falseCount+=1
#                 break
#             if testAttrValue in cur.children.keys():
#                 newNode=cur.children[testAttrValue]
#             else:
#                 newNode=cur.children[cur.children.keys()[0]]
#             stack.append(newNode)
#     print "准确率"
#     print "total",
#     print trueCount+falseCount
#     print "true"
#     print trueCount
#     print float(trueCount)/float(trueCount+falseCount)
    
#     for i in outPutListList:
#         print i
#     '''
#     accuracy
#     (TP+TN)/(P+N)
#     '''
#     numeritor=0
#     denominator=0
#     for i in range(len(outPutListList)):
#         numeritor+=outPutListList[i][i]

#     for i in range(len(outPutListList)):
#         for j in range(len(outPutListList)):
#             denominator+=outPutListList[i][j]
#     print "accuracy is:",
#     print float(numeritor)/float(denominator)
        

preprocess data: 
input list is:  [['2', '3', '13', '2', '7', '4', '11', '3', '11', '2', '11', 0], ['1', '1', '5', '3', '13', '2', '13', '2', '7', '4', '5', 1], ['1', '1', '4', '2', '13', '3', '1', '4', '4', '4', '1', 2], ['1', '2', '11', '3', '9', '1', '11', '4', '8', '3', '8', 3], ['1', '3', '7', '1', '8', '1', '7', '4', '12', '2', '8', 4], ['2', '1', '9', '4', '6', '4', '9', '2', '9', '1', '1', 5], ['2', '4', '10', '2', '1', '4', '11', '3', '11', '2', '11', 6], ['1', '2', '10', '3', '10', '2', '13', '2', '6', '1', '6', 7], ['1', '3', '8', '1', '13', '1', '7', '4', '7', '3', '13', 8], ['1', '1', '11', '1', '10', '4', '10', '3', '13', '1', '13', 9], ['1', '4', '12', '3', '1', '2', '1', '1', '11', '4', '11', 10], ['1', '3', '13', '2', '2', '4', '13', '4', '6', '3', '2', 11], ['2', '1', '13', '3', '12', '2', '12', '1', '9', '4', '12', 12], ['1', '2', '7', '1', '3', '1', '1', '1', '7', '2', '3', 13], ['2', '4', '12', '2', '11', '4', '11', '1', '9', '3', '11', 14], ['1', '3', '9', '1', '1